In [9]:
##크롤링 대상 사이트 : https://www.computer.org (IEEE)

#사용할 라이브러리

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from multiprocessing import Pool, Array, Process
import time


In [10]:
#url 크롤링하는 함수 정의

def croll_link(next_url,link_name):
    req = requests.get('https://www.computer.org'+next_url)
    soup = bs(req.text, 'html.parser')
    next_r_url = soup.select(link_name)
    
    parsed = [] 
    for i in range(0,len(next_r_url)):
        parsed.append(next_r_url[i].get("href")) #url 추출해서 list에 append
    
    return parsed #list 반환

In [11]:
#크롤링 - site에서는 주제-연도-달(2-3달)-내용 으로 되어있음

start = time.time()

# computer science subject link croll
transactionLinks = croll_link('/csdl','a.transactionLink')

# subject -> volume(year)
volumeLinks = croll_link(transactionLinks[0],'a.volumeLink')

# volume -> issue(months)
issueLinks = croll_link(volumeLinks[0],'a.issueLink')

# issue -> contents
abstractLinks = croll_link(issueLinks[0],'div.col-xs-12 > div > div > div > button > a') # contents

# abstractLinks에서 html만 추출 (javascript와 pdf도 태그가 같았기에 soup로 구분하지 못했음)
new_absLinks = []
for i in range(0,len(abstractLinks)):
    if(abstractLinks[i][-8:] == 'abs.html'):
        new_absLinks.append(abstractLinks[i])
        
close = time.time()

print(close-start)


20.603402137756348


In [12]:
abstractLinks = new_absLinks
new_absLinks = []

#abstractLinks

In [13]:
#contents
req = requests.get('https://www.computer.org'+abstractLinks[1])
soup = bs(req.text, 'html.parser')

In [14]:
#title 크롤링
abs_title = (soup.select('div.abstractTitle > a > h2')[0].text) # title 추출
abs_title

'Simulating the Large-Scale Erosion of Genomic Privacy Over Time'

In [15]:
# url에 date내용이 들어있음
abs_yymm = abstractLinks[1][15:22]
abs_yymm

'2018/05'

In [16]:
#contents 크롤링
abs_contents = soup.select('div.abstractText.abstractTextMB')[0].text
#abs_contents


## Data set 늘리기


In [17]:
# Month를 다 합쳐봄 - volume은 같음
# 다 하면 너무 많을 것 같아서 일단 issue(month)만 합쳤음

total_absLink = []
for i in range(0,len(issueLinks)):
    total_absLink += croll_link(issueLinks[i],'div.col-xs-12 > div > div > div > button > a')

#total_absLink

In [18]:
final_absLink = []
for i in range(0,len(total_absLink)):
    if(total_absLink[i][-8:] == 'abs.html'):
        final_absLink.append(total_absLink[i])
#final_absLink

In [22]:
from datetime import datetime

now = str(datetime.now())[0:19]
start = time.time()
def croll_abs(link):
    req = requests.get('https://www.computer.org'+link)
    soup = bs(req.text, 'html.parser')
    journal = {}
    journal['title'] = soup.select('div.abstractTitle > a > h2')[0].text
    journal['abstract'] = soup.select('div.abstractText.abstractTextMB')[0].text
    journal['date'] = link[15:22]
  
    filename = ("./"+"%s"+"_output.txt") % now
    with open(filename, 'a') as f:
    #for data in journal_list:
        o_data = "%s\t%s\t%s\n" % (journal['date'], journal['title'],journal['abstract'])
        f.write(o_data)

pool = Pool(processes=4)
p = pool.map(croll_abs,final_absLink)

close = time.time()
print(close-start)

75.53028893470764
